# Dask Implementation

Implementing the extended collocation (EC) function to out of memory data sets is simple with [`dask`](https://www.dask.org/). All we have to do is use [`map_blocks`](https://docs.dask.org/en/stable/generated/dask.array.map_blocks.html?highlight=map_blocks#dask.array.map_blocks) to map the function to each chunk.

In [ ]:
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar

As an example, let's generate some random larger than memory data in a `dask` array.

In [ ]:
rng = da.random.default_rng()
X = rng.standard_normal((4000, 3, 600, 600), chunks=(-1, -1, 25, 50))
X

Load the EC function from it's associated notebook.

In [ ]:
%run ../TC/EC_function.ipynb

Now, we simply apply it to `map_blocks`. In this, we need to specify `chunks`, `drop_axis`, and `new_axis` as the EC function drops the first and second dimensions and replaces them with a covariance matrix.

In [ ]:
dask_graph = X.map_blocks(
    lambda X: ec_covar_multi(X, corr_sets=[1, 2, 3]),
    dtype=np.float64,
    chunks=(X.chunksize[1], X.chunksize[1]) + X.chunksize[2:],
    drop_axis=[0, 1],
    new_axis=[0, 1],
)
dask_graph

Time to compute the estimates!

In [ ]:
with ProgressBar():
    covar_dask = dask_graph.compute()